# 08 - Hybrid Attention-Based Allocation
This notebook builds on the hybrid strategy logic and integrates interpretability insights from the ATT-LSTM model.
The aim is to:

- Identify if attention weights can inform capital allocation
- Show how interpretability from AI models can enhance investment strategies
- Lay the groundwork for future adaptive allocation via explainable AI (XAI)



## 🎯 Slide 8 – Hybrid Attention-Weighted Strategy

This strategy combines ATT-LSTM predictions with attention-based position sizing. By trading only when attention weights are high, and sizing positions dynamically, it aims to improve interpretability and stability.

We use:
- Binary signals from ATT-LSTM
- Filtering using attention scores
- Dynamic position sizing proportional to attention

This creates a confidence-aware, interpretable AI strategy.


## 1. Imports and Data Load 

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import vectorbt as vbt

In [ ]:
# Load processed predictions (make sure index is datetime)
att_df = pd.read_csv("../data/df_att_with_attention.csv", index_col=0, parse_dates=True)

# Load SPY historical prices
df_spy = pd.read_csv("../data/GSPC_fixed.csv", parse_dates=['Date'], index_col='Date')
df_spy = df_spy[['Adjusted_close']].rename(columns={'Adjusted_close': 'SPY'})

# Align SPY to match att_df date index (business days, holidays included)
df_spy = df_spy.reindex(att_df.index).ffill()  # forward fill to avoid NaNs on missing market holidays



## 2. Signal Generation from Predictions

In [ ]:
# === 2. Signal Generation ===

# Create binary signal from predictions
att_df['signal'] = (att_df['predictions'] > 0.5).astype(int)

# Optional: store raw copies for experimentation
att_df['raw_signal'] = att_df['signal']  # Backup signal
att_df['raw_attention'] = att_df['attention_mean']  # Backup attention weights

# Preview
att_df[['predictions', 'signal', 'attention_mean']].head()



## 3. Attention-Based Weighting

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(att_df.index, att_df['attention_mean'], label='Attention Score', color='crimson')
plt.title("ATT-LSTM Attention Weights Over Time", fontsize=14, fontweight='bold')
plt.xlabel("Date")
plt.ylabel("Attention Weight")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("1_att_lstm_attention_scores.png", dpi=150)
plt.show()


## 4: Attention-Enhanced Allocation Strategy 

### 4.1: Attention-Based Signal Filtering
Only trade if the model is confident (i.e., attention weight exceeds a threshold).

In [ ]:
# Step 1: Skip filtering temporarily
att_df['filtered_signal'] = att_df['signal']

### 4.2: Attention-Based Position Sizing

In [ ]:
# Step 2: Use attention for sizing
att_df['size'] = att_df['filtered_signal'] * att_df['attention_mean']

In [ ]:
att_df['size'].describe()


### 4.3: Backtest Attention-Based Strategy

In [ ]:
# === 4.3 Backtest with Attention-Based Allocation ===
import vectorbt as vbt

pf_att_alloc = vbt.Portfolio.from_signals(
    close=df_spy['SPY'],
    entries=att_df['filtered_signal'] == 1,
    exits=att_df['filtered_signal'] == 0,
    size=att_df['size'],  # fractional sizing
    freq='1D',
    init_cash=100,
    fees=0.001
)


### 4.4: Visualize the Strategy vs Benchmark

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(12, 6))
plt.plot(pf_att_alloc.value(), label='ATT-LSTM + Attention Allocation', linewidth=2)
plt.plot((df_spy['SPY'] / df_spy['SPY'].iloc[0]) * 100, label='SPY Buy & Hold', linestyle='--')
plt.title("Performance: Attention-Weighted Strategy vs SPY", fontsize=14, fontweight='bold')
plt.xlabel("Date")
plt.ylabel("Portfolio Value (Base 100)")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("2_attention_strategy_vs_spy.png", dpi=150)
plt.show()


In [ ]:
att_df['filtered_signal'].value_counts()


In [ ]:
att_df['size'].describe()


In [ ]:
pf_att_alloc.stats()

In [ ]:
att_df['signal'].value_counts()


### Plot: Position Size Over Time
Show how attention affects exposure

In [ ]:
plt.figure(figsize=(12, 4))
plt.plot(att_df.index, att_df['size'], label='Position Size (Attention-Weighted)', color='teal')
plt.title("Position Size Over Time (Driven by Attention)", fontsize=14, fontweight='bold')
plt.xlabel("Date")
plt.ylabel("Fractional Position Size")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("3_attention_position_size.png", dpi=150)
plt.show()


### Stats Summary
Get portfolio stats (trades, drawdown, Sharpe, etc.)

In [ ]:
pf_stats = pf_att_alloc.stats()
print(pf_stats[['Total Trades', 'Total Return [%]', 'Max Drawdown [%]', 'Sharpe Ratio']])


### Plot: Rolling Sharpe
To show consistency of performance

In [ ]:
# Compute 3-month rolling Sharpe Ratio manually (63 trading days)
rolling_returns = pf_att_alloc.daily_returns()
rolling_sharpe = rolling_returns.rolling(window=63).mean() / rolling_returns.rolling(window=63).std()

plt.figure(figsize=(12, 4))
plt.plot(rolling_sharpe, color='purple', label='3-Month Rolling Sharpe')
plt.axhline(0, color='black', linestyle='--', linewidth=0.5)
plt.title("Rolling 3-Month Sharpe Ratio (Attention Strategy)", fontsize=14)
plt.xlabel("Date")
plt.ylabel("Sharpe Ratio")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig("4_rolling_sharpe_attention.png", dpi=150)
plt.show()



In [ ]:
rolling_returns = pf_att_alloc.daily_returns()
sharpe_raw = rolling_returns.rolling(window=63).mean() / rolling_returns.rolling(window=63).std()


In [ ]:
sharpe_annualized = sharpe_raw * np.sqrt(252)  # annualize for daily frequency

In [ ]:
plt.figure(figsize=(12, 5))
plt.plot(sharpe_raw, label='Raw 3-Month Sharpe', linestyle='--', color='gray')
plt.plot(sharpe_annualized, label='Annualized 3-Month Sharpe', color='purple')
plt.axhline(0, color='black', linestyle='--', linewidth=0.5)
plt.title("Rolling 3-Month Sharpe Ratio: Raw vs Annualized", fontsize=14)
plt.xlabel("Date")
plt.ylabel("Sharpe Ratio")
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig("5_rolling_sharpe_comparison.png", dpi=150)
plt.show()
